# Integrating a generic, python tool in pyiron (Level B, recommended approach)
If we need to repeatedly, perform a combination of processes to return a value, a plot, etc., it might make sense to create a Job class for it. Here, we create a simple workflow using available python tools for a specific problem.

Problem: 
we are given as an input the positions of a some particles, plus a energy constant.  
the goal of the workflow: 
- calculate the mean harmonic energy
- return a 3D plot of the cloud of particles.

In [ ]:
from pyiron_base import PythonTemplateJob, DataContainer
import numpy as np
import math
import matplotlib.pyplot as plt

In [ ]:
class harmonicEnergy(PythonTemplateJob):
    def __init__(self, project, job_name):
        super(harmonicEnergy, self).__init__(project, job_name)
        self.input=DataContainer(table_name='inputs')

    def read_input(self, file_name):
        self.input.read(file_name)
    
    def calc_mean_harmonic_energy(self):
        counter=0
        d2=0
        for i,p1 in enumerate(self.input.positions.to_builtin()):
            for p2 in self.input.positions.to_builtin()[i+1:]:
                p2_vec=np.array(p2)
                p1_vec=np.array(p1)
                d = p2_vec - p1_vec
                d_val = math.sqrt(np.dot(d, d))-self.input.equilibrium_lengh
                d2+=d_val**2
                counter+=1
        self.avg_d2=d2/counter
        self.avg_energy=self.input.energy_constant*self.avg_d2
    
    def scatter_plot(self):
        x = [ self.input.positions.to_builtin()[k][0] for k in range(0, len(self.input.positions.to_builtin()))]
        y = [ self.input.positions.to_builtin()[k][1] for k in range(0, len(self.input.positions.to_builtin()))]
        z = [ self.input.positions.to_builtin()[k][1] for k in range(0,len(self.input.positions.to_builtin()))]
        fig = plt.figure()
        ax = fig.add_subplot(projection='3d')
        ax.scatter(x, y, z, marker='o')

    # This function is executed 
    def run_static(self):
        self.calc_mean_harmonic_energy()
        with self.project_hdf5.open("output/generic") as h5out: 
             h5out["avg_harmonic_energy"] = self.avg_energy
        self.status.finished = True

## Exercise 1:
Create a project and a job of type harmonicEnergy, which reads in `input.yml` and calculates the mean harmonic energy. Also create the 3d scatter plot of the particles.

## Exercise 2:  
How to access the `energy_tot` from the output of the job. 